In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_data=pd.read_csv("/content/drive/MyDrive/ML_Assignment2/netflix/TrainingRatings.txt", names=["MovieID", "UserID", "Rating"])
train_data.head()

,MovieID,UserID,Rating
0,8,1744889,1.0
1,8,1395430,2.0
2,8,1205593,4.0
3,8,1488844,4.0
4,8,1447354,1.0


In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3255352 entries, 0 to 3255351
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   MovieID  int64  
 1   UserID   int64  
 2   Rating   float64
dtypes: float64(1), int64(2)
memory usage: 74.5 MB


In [6]:
user_item_df=pd.pivot_table(train_data, values="Rating", index="UserID", columns="MovieID").fillna(0)
user_item_df.head()

MovieID,8,28,43,48,61,64,66,92,96,111,122,123,127,140,145,154,156,174,185,192,207,214,218,222,229,237,259,267,276,287,305,318,323,336,359,361,380,395,398,409,...,17337,17338,17344,17348,17358,17394,17411,17423,17447,17454,17466,17515,17522,17523,17534,17536,17551,17554,17556,17558,17561,17574,17616,17624,17626,17635,17640,17642,17650,17653,17654,17660,17689,17693,17706,17725,17728,17734,17741,17742
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
test_data=pd.read_csv("/content/drive/MyDrive/ML_Assignment2/netflix/TestingRatings.txt", names=["MovieID", "UserID", "Rating"])
test_data.head()

,MovieID,UserID,Rating
0,8,573364,1.0
1,8,2149668,3.0
2,8,1089184,3.0
3,8,2465894,3.0
4,8,534508,1.0


In [15]:
# make a copy of train and test datasets
dummy_train = train_data.copy()
dummy_test = test_data.copy()

dummy_train['Rating'] = dummy_train['Rating'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['Rating'] = dummy_test['Rating'].apply(lambda x: 1 if x > 0 else 0)

In [16]:
# The movies not rated by user is marked as 1 for prediction 
dummy_train = dummy_train.pivot(index = 'UserID', columns = 'MovieID', values = 'Rating').fillna(1)

# The movies not rated by user is marked as 0 for evaluation 
dummy_test = dummy_test.pivot(index ='UserID', columns = 'MovieID', values = 'Rating').fillna(0)

In [8]:
dummy_train.head()

MovieID,8,28,43,48,61,64,66,92,96,111,122,123,127,140,145,154,156,174,185,192,207,214,218,222,229,237,259,267,276,287,305,318,323,336,359,361,380,395,398,409,...,17337,17338,17344,17348,17358,17394,17411,17423,17447,17454,17466,17515,17522,17523,17534,17536,17551,17554,17556,17558,17561,17574,17616,17624,17626,17635,17640,17642,17650,17653,17654,17660,17689,17693,17706,17725,17728,17734,17741,17742
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
79,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
199,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
481,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
769,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [9]:
dummy_test.head()

MovieID,8,28,43,48,61,64,66,92,96,111,122,123,127,145,154,156,174,185,192,207,214,218,222,229,237,259,267,276,287,305,318,323,336,359,361,380,395,398,409,417,...,17334,17337,17338,17344,17348,17358,17394,17411,17423,17447,17454,17466,17515,17522,17523,17534,17536,17551,17554,17556,17558,17561,17574,17616,17624,17626,17635,17640,17642,17650,17653,17654,17689,17693,17706,17725,17728,17734,17741,17742
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:

# User Similarity Matrix using Cosine similarity as a similarity measure between Users
user_similarity = cosine_similarity(user_item_df)
user_similarity[np.isnan(user_similarity)] = 0
print(user_similarity)
print(user_similarity.shape)

[[1.         0.41957691 0.39019949 ... 0.37195276 0.37758772 0.46980239]
 [0.41957691 1.         0.36185935 ... 0.22611836 0.35119518 0.41948276]
 [0.39019949 0.36185935 1.         ... 0.39178429 0.30393094 0.29161915]
 ...
 [0.37195276 0.22611836 0.39178429 ... 1.         0.22180705 0.23167963]
 [0.37758772 0.35119518 0.30393094 ... 0.22180705 1.         0.41185017]
 [0.46980239 0.41948276 0.29161915 ... 0.23167963 0.41185017 1.        ]]
(28978, 28978)


## **Predicting user rating on movies**

In [11]:
user_predicted_ratings = np.dot(user_similarity, user_item_df)
user_predicted_ratings

array([[ 3341.43763035, 18591.47966584,    42.17591148, ...,
           38.30482378,  1136.24922938,   249.85555121],
       [ 2773.97934545, 17298.28155045,    36.69013861, ...,
           29.87368244,   986.38070373,   226.81959826],
       [ 3066.32042888, 16093.84008091,    38.57984824, ...,
           31.67996951,  1123.00524534,   199.024523  ],
       ...,
       [ 2955.71888145, 13777.28181744,    35.56452708, ...,
           35.71970978,  1025.045092  ,   188.76975814],
       [ 2581.52945855, 15348.31543553,    31.50126594, ...,
           27.89067041,   915.89882898,   201.05502611],
       [ 2964.29837175, 18709.48699359,    40.29063791, ...,
           30.75150511,  1107.17348327,   240.07688208]])

In [12]:
# np.multiply for cell-by-cell multiplication 

user_final_ratings = np.multiply(user_predicted_ratings, dummy_train)
user_final_ratings.head()

MovieID,8,28,43,48,61,64,66,92,96,111,122,123,127,140,145,154,156,174,185,192,207,214,218,222,229,237,259,267,276,287,305,318,323,336,359,361,380,395,398,409,...,17337,17338,17344,17348,17358,17394,17411,17423,17447,17454,17466,17515,17522,17523,17534,17536,17551,17554,17556,17558,17561,17574,17616,17624,17626,17635,17640,17642,17650,17653,17654,17660,17689,17693,17706,17725,17728,17734,17741,17742
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,0.000000,0.000000,42.175911,2233.083740,46.505091,37.528993,44.376919,67.087927,150.109593,10607.567359,1536.138179,105.646483,1863.785821,35.620666,133.911034,48.409082,4171.034624,122.208099,0.000000,91.346787,55.615538,66.371470,32.702242,31.158196,242.666692,189.278275,258.758393,726.657520,40.329518,65.722707,9053.684649,39.323741,382.821735,100.589346,2876.704513,9743.437741,347.513692,1381.119923,890.634860,291.309301,...,39.354248,3752.534338,130.111848,605.539682,0.000000,96.371417,4923.919076,2711.554196,249.070358,234.010635,414.928204,147.030496,630.503577,92.313126,31.980939,2283.805235,59.313491,588.528127,299.200711,0.000000,1191.926295,9543.374677,61.003861,2866.722261,75.427067,240.738240,94.511243,1266.686577,902.244194,2098.622701,3293.514072,24.113473,23.657824,2907.519282,122.694492,2663.361944,60.760389,38.304824,1136.249229,249.855551
79,2773.979345,17298.281550,36.690139,2016.200312,43.229528,35.492524,39.796562,59.945339,134.946266,10113.535373,1325.721413,94.902245,1738.982672,35.514041,101.077028,42.337955,3266.758190,113.243333,1759.465424,81.552147,52.868614,64.858675,28.365119,26.836695,201.886893,135.145780,246.868035,742.150743,35.929778,59.262702,8820.351764,34.551429,339.387768,79.285735,2431.590089,0.000000,319.295825,1138.713673,715.431411,250.458100,...,37.926663,3503.403840,105.370388,560.775484,7533.922185,84.349652,4219.874121,2571.822299,203.252618,198.468951,348.551668,144.909435,526.724967,75.262447,28.158352,1995.261855,51.997939,544.209696,260.627505,9602.170337,1158.499906,9028.845216,47.852730,2939.838228,59.039599,224.653941,88.197244,1175.372723,758.292056,1945.049909,3167.219520,21.114533,21.179363,2811.012084,106.857162,2292.096188,46.785583,29.873682,986.380704,226.819598
199,3066.320429,16093.840081,38.579848,2063.830653,39.364845,34.826102,40.237528,68.786535,139.285338,0.000000,1498.843368,108.054333,1607.093035,29.141352,116.171102,46.571355,3755.505603,106.717693,1775.082129,83.773441,49.500117,55.351280,33.886176,29.260375,231.480819,157.508471,245.858931,555.183489,34.321358,60.628369,7898.579736,34.131756,364.313941,99.093928,2521.352919,8388.424721,318.726430,1153.999237,853.990771,267.830539,...,36.303130,3202.095399,118.129624,523.670055,8880.127258,85.312680,3891.564741,2361.966222,222.568533,230.582103,430.949274,124.463169,487.485372,68.466298,30.883591,2000.315094,57.673410,504.737423,251.865366,10282.878879,1012.139985,8382.618510,49.492733,2324.338637,79.992257,211.551700,83.752434,1154.650341,788.225309,1910.131398,2880.122515,19.038235,22.254124,2475.613009,118.284718,2629.897871,56.482550,31.679970,1123.005245,199.024523
481,2592.534547,16111.845425,34.657836,1953.309163,40.089194,32.462078,36.600857,65.391762,121.898047,0.000000,1374.143531,97.703904,1539.492830,30.875672,85.738227,43.919122,2799.415560,98.084770,1583.117806,83.107616,48.485631,60.539411,27.181260,25.774258,188.057754,117.687044,236.465079,625.463679,31.799865,55.685526,8205.761111,30.623649,345.198394,69.580193,2263.347467,8386.905735,283.400684,933.064529,659.915619,233.538612,...,34.471420,3086.634282,98.434593,524.724877,7317.863096,75.426346,3524.868031,0.000000,184.581740,207.418102,338.965248,129.323201,406.314363,60.365418,26.402475,1750.536309,50.993312,492.117484,231.087867,9425.084754,1088.718750,0.000000,38.120226,2586.435203,62.859959,208.954356,86.370861,1151.295931,666.578242,1856.957061,2866.198175,18.081066,19.570702,2540.773326,109.863612,2204.653316,42.977313,25.980585,1043.299604,195.22

In [8]:
user_item_test_df=pd.pivot_table(test_data, values="Rating", index="UserID", columns="MovieID").fillna(0)
test_user_similarity = cosine_similarity(user_item_test_df)
test_user_similarity[np.isnan(test_user_similarity)] = 0

print(test_user_similarity)

print(test_user_similarity.shape)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
(27555, 27555)


In [9]:
user_predicted_ratings_test = np.dot(test_user_similarity, user_item_test_df)
user_predicted_ratings_test

array([[ 3.84682955,  5.88114317,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 5.2824231 , 13.63369665,  0.        , ...,  0.        ,
         0.73284578,  0.        ],
       [ 1.56549132, 13.61924128,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 2.05468302, 13.5284027 ,  0.        , ...,  0.        ,
         2.36623487,  0.        ],
       [ 5.43083768, 21.83047084,  0.        , ...,  0.        ,
         1.4720338 ,  0.91652764],
       [ 1.34588189,  8.69301276,  0.        , ...,  0.        ,
         0.76167761,  1.02597835]])

In [17]:
test_user_final_rating = np.multiply(user_predicted_ratings_test, dummy_test)
test_user_final_rating.head()

MovieID,8,28,43,48,61,64,66,92,96,111,122,123,127,145,154,156,174,185,192,207,214,218,222,229,237,259,267,276,287,305,318,323,336,359,361,380,395,398,409,417,...,17334,17337,17338,17344,17348,17358,17394,17411,17423,17447,17454,17466,17515,17522,17523,17534,17536,17551,17554,17556,17558,17561,17574,17616,17624,17626,17635,17640,17642,17650,17653,17654,17689,17693,17706,17725,17728,17734,17741,17742
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
X = test_user_final_rating.copy() 
X = X[X > 0] # only consider non-zero values as 0 means the user haven't rated the movies

scaler = MinMaxScaler(feature_range = (0.5, 5))
scaler.fit(X)
pred = scaler.transform(X)

print(pred)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [20]:
# total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(pred))
total_non_nan

100478

In [21]:
test=pd.pivot_table(test_data, values="Rating", index="UserID", columns="MovieID")
test.head()

MovieID,8,28,43,48,61,64,66,92,96,111,122,123,127,145,154,156,174,185,192,207,214,218,222,229,237,259,267,276,287,305,318,323,336,359,361,380,395,398,409,417,...,17334,17337,17338,17344,17348,17358,17394,17411,17423,17447,17454,17466,17515,17522,17523,17534,17536,17551,17554,17556,17558,17561,17574,17616,17624,17626,17635,17640,17642,17650,17653,17654,17689,17693,17706,17725,17728,17734,17741,17742
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# RMSE Score

diff_sqr_matrix = (test - pred)**2
sum_of_squares_err = diff_sqr_matrix.sum().sum() # df.sum().sum() by default ignores null values

rmse = np.sqrt(sum_of_squares_err/total_non_nan)
print(rmse)

1.3975080554168327


In [23]:
# Mean abslute error

mae = np.abs(pred - test).sum().sum()/total_non_nan
print(mae)

1.176214030356687
